In [0]:
!wget 1k8G5bOTWHTlV7UWAUHLZjpGP-yy4iyeh

In [0]:
https://drive.google.com/file/d/1qPnDc3SJ473bqfATE4AfkdkWOQkud6e1/view?usp=sharing
https://drive.google.com/file/d/1BkJtKyrehYTeW_Meul-jhTglhNpo2A9H/view?usp=sharing

In [0]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1k8G5bOTWHTlV7UWAUHLZjpGP-yy4iyeh' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1k8G5bOTWHTlV7UWAUHLZjpGP-yy4iyeh" -O unlabeled_10M_1.tsv && rm -rf /tmp/cookies.txt

In [0]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1qPnDc3SJ473bqfATE4AfkdkWOQkud6e1' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1qPnDc3SJ473bqfATE4AfkdkWOQkud6e1" -O dev_labeled.tsv && rm -rf /tmp/cookies.txt


In [0]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1BkJtKyrehYTeW_Meul-jhTglhNpo2A9H' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1BkJtKyrehYTeW_Meul-jhTglhNpo2A9H" -O train_labeled.tsv && rm -rf /tmp/cookies.txt


--2020-04-23 00:00:05--  https://docs.google.com/uc?export=download&confirm=&id=1BkJtKyrehYTeW_Meul-jhTglhNpo2A9H
Resolving docs.google.com (docs.google.com)... 172.217.203.101, 172.217.203.113, 172.217.203.102, ...
Connecting to docs.google.com (docs.google.com)|172.217.203.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0o-ag-docs.googleusercontent.com/docs/securesc/nbi086lor53vr0ukqaiq5ld3lni5ruic/0t094896m7t3vejj08felvpsn9vn01fl/1587600000000/12684776072375856457/11715250907945946374Z/1BkJtKyrehYTeW_Meul-jhTglhNpo2A9H?e=download [following]
--2020-04-23 00:00:05--  https://doc-0o-ag-docs.googleusercontent.com/docs/securesc/nbi086lor53vr0ukqaiq5ld3lni5ruic/0t094896m7t3vejj08felvpsn9vn01fl/1587600000000/12684776072375856457/11715250907945946374Z/1BkJtKyrehYTeW_Meul-jhTglhNpo2A9H?e=download
Resolving doc-0o-ag-docs.googleusercontent.com (doc-0o-ag-docs.googleusercontent.com)... 173.194.216.132, 2607:f8b0:400c:c12::84
Connectin

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
import re
import time
import pickle
import os
import pandas as pd
from collections import Counter
pd.set_option('display.max_colwidth', 200)

# LOAD NADI DATASET

In [0]:
data_train0 = pd.read_csv("/content/drive/My Drive/unlabeled tweets/unlabeled_10M_1.tsv",delimiter='\t',encoding='utf-8')
for i in range(2,9):
  data_train0 = data_train0.append(pd.read_csv("/content/drive/My Drive/unlabeled tweets/unlabeled_10M_"+str(i)+".tsv",delimiter='\t',encoding='utf-8'))

In [0]:
data_train0.info()

In [0]:
len(data_train0)

2937600

In [0]:
# Un labeled
data_train = pd.read_csv("/content/drive/My Drive/unlabeled_10M.tsv",delimiter='\t',encoding='utf-8')
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 698000 entries, 0 to 697999
Data columns (total 2 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   #1 tweet_ID       698000 non-null  int64 
 1   #2 tweet_content  698000 non-null  object
dtypes: int64(1), object(1)
memory usage: 10.7+ MB


In [0]:
import numpy as np
import pandas as pd

# unlabeled = pd.read_csv("/content/drive/My Drive/unlabeled_10M.tsv",delimiter='\t',encoding='utf-8')
dataset = pd.read_csv("/content/drive/My Drive/train_labeled.tsv",delimiter='\t',encoding='utf-8')

dev_labeled = pd.read_csv("/content/drive/My Drive/dev_labeled.tsv",delimiter='\t',encoding='utf-8')
dev_length = len(dev_labeled)
dataset = dataset.append(dev_labeled)

dataset.info()
new_names={"#3 country_label":'country_label',"#4 province_label":"province_label","#1 tweet_ID":"id","#2 tweet_content":"tweet"}
dataset=dataset.rename(columns=new_names)



# Convert labels into integers
label_task1 = dataset.country_label
d = dict(zip(np.unique(label_task1), range(0,21)))
labels_ints = dataset['country_label'].map(d, na_action='ignore')


# Splitting
tweets = dataset['tweet'].values
countries = labels_ints

train_tweets = tweets[:] 
train_labels = countries[:]

'''val_tweets = tweets[16000:20000]
val_labels = countries[16000:20000]

test_tweets = tweets[20000:]
test_labels = countries[20000:]
train_tweets.shape, val_tweets.shape, test_tweets.shape

# import pandas as pd 
import pandas as pd 
# Calling DataFrame constructor on list 
df = pd.DataFrame(val_labels) 
print(df)
df.to_csv('Step1.csv',index=False)
df = pd.read_csv('Step1.csv')
val_labels = df['country_label']'''

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25957 entries, 0 to 4956
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   #1 tweet_ID        25957 non-null  object
 1   #2 tweet_content   25957 non-null  object
 2   #3 country_label   25957 non-null  object
 3   #4 province_label  25957 non-null  object
dtypes: object(4)
memory usage: 1013.9+ KB


"val_tweets = tweets[16000:20000]\nval_labels = countries[16000:20000]\n\ntest_tweets = tweets[20000:]\ntest_labels = countries[20000:]\ntrain_tweets.shape, val_tweets.shape, test_tweets.shape\n\n# import pandas as pd \nimport pandas as pd \n# Calling DataFrame constructor on list \ndf = pd.DataFrame(val_labels) \nprint(df)\ndf.to_csv('Step1.csv',index=False)\ndf = pd.read_csv('Step1.csv')\nval_labels = df['country_label']"

In [0]:
len(train_tweets)

25957

# Text Cleaning and Preprocessing

In [0]:
# coding=utf-8
import re
import string
import sys

arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

def remove_links(text):
  text = re.sub(r'http\S+', '', text)
  return text

def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

def remove_english(text):
  text = re.sub("[a-zA-Z]",'',text)
  return text
def remove_diacritics(text):
    text = re.sub(arabic_diacritics, '', text)
    return text


def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)


def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

def pre_process_document(text):
  text = remove_links(text)
  text = remove_punctuations(text)
  text = remove_diacritics(text)
  text = remove_repeating_char(text)
  text = remove_english(text)
  return text

pre_process_corpus = np.vectorize(pre_process_document)



In [0]:
# CLEAN FOR ELMO UNLABELED DATA
data_train0['#2 tweet_content'] = pre_process_corpus(data_train0['#2 tweet_content'])

In [0]:
# CLEAN FOR ELMO UNLABELED DATA
data_train['#2 tweet_content'] = pre_process_corpus(data_train['#2 tweet_content'])

In [0]:
print(data_train['#2 tweet_content'][697999])

RT Almajlis وزير التربية والنائب دمحمد الحويلة في حفل تكريم الخريجين من الطلبة والطالبات الكويتين الدارسين في الجامعات المصرية 


In [0]:
train_tweets = pre_process_corpus(train_tweets)
# val_tweets = pre_process_corpus(val_tweets)
# test_tweets = pre_process_corpus(test_tweets)
#train_tweets = pre_process_corpus(tweets)

In [0]:
data_train = data_train0[:1000000]

99999                                                                                                   85 شن في خيرها الخمس .
99999    بلادي بلادي لكي حبي و فؤادي بلادي أرض الكنانة أرض الأمن و الأمان أرض السلام أرض الأديان أرض الجدعان أرض الثقافه أر  .
99999              اقترب رمضان ل يضيء لنا شهرا كاملا متلىء ب الأجواء الإيمانيه و المطمئنه و يارب أدخله علينا وأنت راض عنا❤🌙🕊 .
99999                                                     كل الي بيحصل دا لازم يقف كفايا دم كفايا الي ماتوا يارب عديها ع خير .
Name: #2 tweet_content, dtype: object

# Prepare Training Data

In [0]:
print("Shape of training data = ", data_train.shape)
# data_train.sample(10)

# Adding white space separated full stop to each sentence in data.
data_train['#2 tweet_content'] = data_train['#2 tweet_content'] + " ."
data_train_ = data_train['#2 tweet_content']#[1000:3000]#[300000:]
print(data_train_)

# As the training requires multiple files with one text sentence per line,
# we will create 20K training files by writing 6 sentences per file. After running the below python snippet, we get 20K files in train directory.
if not os.path.exists("/content/ELMO_TRAIN_FILES3/train/"):
    os.makedirs("/content/ELMO_TRAIN_FILES3/train/")
 
for i in range(0,data_train_.shape[0],10):
    text = "\n".join(data_train_[i:i+10].tolist())
    fp = open("/content/ELMO_TRAIN_FILES3/train/"+str(i)+".txt","w")
    fp.write(text)
    fp.close()

Shape of training data =  (698000, 2)
0                                                   من الذى سيحد قيمة الأصول التى ستحول ملكيتها لشعب النظيف او الشريف او المسرور او المعزلدين الوطنى .
1                                                                                                     هنا طلع عالم اخر مليء باشياء حلوه معكم على طول .
2                            السعودية تعاني من أزمة هوية وأزمة إنجاز وكل الإنجازات التي تسمعون بها هي وهمية والسب في ذلك في رأي هو العبودية لآل سعود .
3                                                                                                أصبح سقوط الملكة قاب قوسين قصيدة نبطية رائعة وقوية  .
4         واستبد الامراء والاميرات على الشعب وعاثت اسرة ال سعود فسادا فدمرت الاقتصاد ونهبت موارد الدولة واصبحت السعودية اكبر دولة مديونة على وجة الا .
                                                                              ...                                                                     
697995                                                  

In [0]:
data_train_[299995]

'UNAVAILABLE .'

# Preparing Validation Data


In [0]:
#data_dev = pd.read_csv("swb/swb-dev.csv")
# ELMO_TRAIN_FILES/valid
data_dev = data_train#['#2 tweet_content'] = data_dev['#2 tweet_content'] + " ."

data_dev = data_train['#2 tweet_content'][200000:250000]
if not os.path.exists("ELMO_VALID_FILES1/dev/"):
    os.makedirs("ELMO_VALID_FILES1/dev/")
 
for i in range(0,data_dev.shape[0],10):
    text = "\n".join(data_dev[i:i+10].tolist())
    fp = open("ELMO_VALID_FILES1/dev/"+str(i)+".txt","w")
    fp.write(text)
    fp.close()

In [0]:
data_train_ = data_train['#2 tweet_content'][:3000]#[300000:]

# Preparing Vocab Files
The vocabulary file is a a text file with one token per line. It must also include the special tokens <S>, </S> and <UNK> (case sensitive) in the file. The vocabulary file should be sorted in descending order by token count in your training data. The first three lines should be the special tokens (<S>, </S> and <UNK>), then the most common token in the training data, ending with the least common token.

In [0]:
texts = " ".join(data_train_.tolist())
words = texts.split(" ")
print("Number of tokens in Training data = ",len(words))
dictionary = Counter(words)
print("Size of Vocab",len(dictionary))
sorted_vocab = ["<S>","</S>","<UNK>"]
sorted_vocab.extend([pair[0] for pair in dictionary.most_common()])
 
text = "\n".join(sorted_vocab)
fp = open("/content/ELMO_TRAIN_FILES3/vocab.txt","w")
fp.write(text)
fp.close()


Number of tokens in Training data =  10744697
Size of Vocab 688509


In [0]:
sorted_vocab

# Clone ELMO MODEL From https://github.com/allenai/bilm-tf
And Start Training

In [0]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
!git clone https://github.com/allenai/bilm-tf

In [0]:
%cd '/content/drive/My Drive/Train_ELMO/bilm-tf/bin'

/content/drive/My Drive/Train_ELMO/bilm-tf/bin


In [0]:
# Download options.json file
%cd '/content/drive/My Drive/Train_ELMO/checkpoint/'
!wget https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x1024_128_2048cnn_1xhighway/elmo_2x1024_128_2048cnn_1xhighway_options.json

In [0]:
# Training !!!!
!python /content/drive/My\ Drive/Train_ELMO/bilm-tf/bin/train_elmo.py --train_prefix='/content/ELMO_TRAIN_FILES3/train/*' --vocab_file '/content/ELMO_TRAIN_FILES3/vocab.txt' --save_dir '/content/drive/My Drive/Train_ELMO/checkpoint/'
# Get weights file
!python /content/drive/My\ Drive/Train_ELMO/bilm-tf/bin/dump_weights.py --save_dir '/content/drive/My Drive/Train_ELMO/checkpoint/' --outfile '/content/drive/My Drive/Train_ELMO/ELMO_weights_1Million_SentPerFile_Arabic.hdf5'



Found 69800 shards at /content/ELMO_TRAIN_FILES3/train/*
Loading data from: /content/ELMO_TRAIN_FILES3/train/556630.txt
Loaded 10 sentences.
Finished loading
Found 69800 shards at /content/ELMO_TRAIN_FILES3/train/*
Loading data from: /content/ELMO_TRAIN_FILES3/train/436230.txt
Loaded 10 sentences.
Finished loading






Instructions for updating:
Use the `axis` argument instead
USING SKIP CONNECTIONS
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:


In [0]:
# Restart - Training !!!!
!python /content/drive/My\ Drive/Train_ELMO/bilm-tf/bin/restart.py --train_prefix='/content/ELMO_TRAIN_FILES3/train/*' --vocab_file '/content/ELMO_TRAIN_FILES1/vocab.txt' --save_dir '/content/drive/My Drive/Train_ELMO/checkpoint/'
# Get weights file
#!python /content/drive/My\ Drive/Train_ELMO/bilm-tf/bin/dump_weights.py --save_dir '/content/drive/My Drive/Train_ELMO/checkpoint/' --outfile '/content/drive/My Drive/Train_ELMO/ELMO_weights700k_10SentPerFile_Arabic.hdf5'

In [0]:
!python /content/drive/My\ Drive/Train_ELMO/bilm-tf/bin/dump_weights.py --save_dir '/content/drive/My Drive/Train_ELMO/checkpoint/' --outfile '/content/drive/My Drive/Train_ELMO/ELMO_weights_test_weigths.hdf5'

In [0]:
# Evaluating!!! ( SKIP )
!python /content/drive/My\ Drive/Train_ELMO/bilm-tf/bin/run_test.py --test_prefix='/content/ELMO_VALID_FILES2/dev/*' --vocab_file '/content/ELMO_TRAIN_FILES2/vocab.txt' --save_dir='/content/drive/My Drive/Train_ELMO/checkpoint/'

# Convert Tf Checkpoint to hdf5

This step is important to convert the checkpoint model to hdf5 format which can be easily used for inference.

In [0]:
# Get weights file
!python /content/drive/My\ Drive/Train_ELMO/bilm-tf/bin/dump_weights.py --save_dir '/content/drive/My Drive/Train_ELMO/checkpoint/' --outfile '/content/drive/My Drive/Train_ELMO/ELMO_weights200k_Arabic.hdf5'

# Prediction

Now,

    Keep the dumped weights file in newly created model folder.
    Create an options.json file for the newly trained model in same folder.
    It is important to always set n_characters to 262 after training.
    Keep vocab.txt in model directory.


In [0]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import numpy as np
import scipy.spatial.distance as ds
from data import Batcher 
from model import BidirectionalLanguageModel
from elmo import weight_layers
 
# Location of pretrained LM.  Here we use the test fixtures.
datadir = os.path.join('/content/drive/My Drive/', 'Train_ELMO')
vocab_file = os.path.join(datadir, 'vocab.txt')
options_file = os.path.join(datadir+'/checkpoint/', 'options.json')
weight_file = os.path.join(datadir, 'ELMO_weights_test_weigths.hdf5')
# Create a Batcher to map text to character ids.
batcher = Batcher(vocab_file, 50)
 
# Input placeholders to the biLM.
context_character_ids = tf.placeholder('int32', shape=(None, None, 50))
 
# Build the biLM graph.
bilm = BidirectionalLanguageModel(options_file, weight_file)
 
# Get ops to compute the LM embeddings.
context_embeddings_op = bilm(context_character_ids)
 
# Get an op to compute ELMo (weighted average of the internal biLM layers)
elmo_context_input = weight_layers('input', context_embeddings_op, l2_coef=0.0)

tokenized_context = [sentence.split() for sentence in train_tweets]
'''
# Now we can compute embeddings.
raw_context = ['Technology has advanced so much in new scientific world',
                'My child participated in fancy dress competition',
                'Fashion industry has seen tremendous growth in new designs']
'''

# print(tokenized_context)



Instructions for updating:
Use the `axis` argument instead
USING SKIP CONNECTIONS
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






"\n# Now we can compute embeddings.\nraw_context = ['Technology has advanced so much in new scientific world',\n                'My child participated in fancy dress competition',\n                'Fashion industry has seen tremendous growth in new designs']\n"

In [0]:
raw_context2 = ['صباح الفل يزميلي',
                'مسا مسا يا ابراهييم خلصانة بشياكة',
                'ايه تاه ايه تاه الحقو']

tokenized_context2 = [sentence.split() for sentence in raw_context2]
print(tokenized_context2)

[['صباح', 'الفل', 'يزميلي'], ['مسا', 'مسا', 'يا', 'ابراهييم', 'خلصانة', 'بشياكة'], ['ايه', 'تاه', 'ايه', 'تاه', 'الحقو', 'يا', 'بنات', 'اه', 'يبن', 'ال']]


In [0]:
tokenized_context.shape

In [0]:
elmo_context_input

{'regularization_op': [<tf.Tensor 'input_ELMo_W/Regularizer/mul:0' shape=() dtype=float32>],
 'weighted_op': <tf.Tensor 'mul_3:0' shape=(?, ?, 256) dtype=float32>}

In [0]:
context_ids.shape

(16000, 64, 50)

In [0]:
def compute_embds(sess,context_ids__):
   return sess.run(
        elmo_context_input['weighted_op'],
        feed_dict={context_character_ids: context_ids__} )

In [0]:
# Train DATA
with tf.Session() as sess:
    # It is necessary to initialize variables once before running inference.
    sess.run(tf.global_variables_initializer())
 
    # Create batches of data.
    context_ids = batcher.batch_sentences(tokenized_context)
    print("Shape of context ids = ", context_ids.shape)
    context_ids_1 = [context_ids[i:i+100] for i in range(0,context_ids.shape[0],100)]

    # context_ids_1 = context_ids[:100]
    # Compute ELMo representations (here for the input only, for simplicity).
    elmo_context_input_ = [compute_embds(sess, x) for x in context_ids_1]
    
    elmo_context_input_ = np.concatenate(elmo_context_input_, axis = 0)
print("Shape of generated embeddings = ",elmo_context_input_.shape)

Shape of context ids =  (25957, 64, 50)
Shape of generated embeddings =  (25957, 62, 256)


In [0]:
elmo_context_input_ = np.reshape(elmo_context_input_,(25957,62*256))

In [0]:
print(len(elmo_context_input_))
print(len(train_labels))

25957
25957


# Skip these

In [0]:
# ( SKIP )
#Validation tweets
tokenized_context_val = [sentence.split() for sentence in val_tweets]
#print(tokenized_context_test)
with tf.Session() as sess:
    # It is necessary to initialize variables once before running inference.
    sess.run(tf.global_variables_initializer())
 
    # Create batches of data.
    context_ids_val = batcher.batch_sentences(tokenized_context_val)
    print("Shape of context ids = ", context_ids_val.shape)
    context_ids_2 = [context_ids_val[i:i+100] for i in range(0,context_ids_val.shape[0],100)]

    # context_ids_1 = context_ids[:100]
    # Compute ELMo representations (here for the input only, for simplicity).
    elmo_context_input_val = [compute_embds(sess, x) for x in context_ids_2]
    elmo_context_input_val = np.concatenate(elmo_context_input_val, axis = 0)
print("Shape of generated embeddings = ",elmo_context_input_val.shape)

Shape of context ids =  (4000, 62, 50)
Shape of generated embeddings =  (4000, 60, 256)


In [0]:
# ( SKIP )
tokenized_context_test = [sentence.split() for sentence in test_tweets]
#print(tokenized_context_test)
with tf.Session() as sess:
    # It is necessary to initialize variables once before running inference.
    sess.run(tf.global_variables_initializer())
 
    # Create batches of data.
    context_ids_test = batcher.batch_sentences(tokenized_context_test)
    print("Shape of context ids = ", context_ids_test.shape)
    context_ids_3 = [context_ids_test[i:i+100] for i in range(0,context_ids_test.shape[0],100)]

    # context_ids_1 = context_ids[:100]
    # Compute ELMo representations (here for the input only, for simplicity).
    elmo_context_input_test = [compute_embds(sess, x) for x in context_ids_3]
    elmo_context_input_test = np.concatenate(elmo_context_input_test, axis = 0)
print("Shape of generated embeddings = ",elmo_context_input_test.shape)

Shape of context ids =  (1000, 60, 50)
Shape of generated embeddings =  (1000, 58, 256)


In [0]:
print(elmo_context_input_.shape)
print(elmo_context_input_val.shape)
print(elmo_context_input_test.shape)

(16000, 62, 256)
(4000, 60, 256)
(1000, 58, 256)


In [0]:
# Reshaping
elmo_context_input_ = np.reshape(elmo_context_input_,(16000,62*256))
elmo_context_input_val = np.reshape(elmo_context_input_val,(4000, 60*256))
elmo_context_input_test = np.reshape(elmo_context_input_test, (1000, 58*256))

In [0]:
print(np.shape(elmo_context_input_))

In [0]:
elmo_context_input_

In [0]:
# Computing euclidean distance between words embedding
euc_dist_bet_tech_computer = np.linalg.norm(elmo_context_input_[1,5,:]-elmo_context_input_[0,0,:])
euc_dist_bet_computer_fashion = np.linalg.norm(elmo_context_input_[1,5,:]-elmo_context_input_[2,0,:])
# Computing cosine distance between words embedding
cos_dist_bet_tech_computer = ds.cosine(elmo_context_input_[1,5,:],elmo_context_input_[0,0,:])
cos_dist_bet_computer_fashion = ds.cosine(elmo_context_input_[1,5,:],elmo_context_input_[2,0,:])
 
print("Euclidean Distance Comparison - ")
print("\nDress-Technology = ",np.round(euc_dist_bet_tech_computer,2),"\nDress-Fashion = ",
      np.round(euc_dist_bet_computer_fashion,2))
print("\n\nCosine Distance Comparison - ")
print("\nDress-Technology = ",np.round(cos_dist_bet_tech_computer,2),"\nDress-Fashion = ",
      np.round(cos_dist_bet_computer_fashion,2))

# Classification

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
# Split data back to train and valid
X_train, X_test, y_train, y_test = train_test_split(elmo_context_input_, train_labels, test_size=0.2)
'''X_train = elmo_context_input_[:21000]
y_train = train_labels[:21000]
X_test = elmo_context_input_[21000:]
y_test = train_labels[21000:]'''
print("Splitted.")
lreg = LogisticRegression()
lreg.fit(X_train, y_train)
print("trained")
preds = lreg.predict(X_test)
#preds_valid = lreg.predict(xvalid)
print("Acc = ", accuracy_score(preds, y_test))
print("F1-Macro",f1_score(y_test, preds,average='macro'))

Splitted.


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


trained
Acc =  0.21302003081664098
F1-Macro 0.05349712811582373


In [0]:
print(f1_score(y_test, preds,average='micro'))

0.2235714285714286


In [0]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import MultinomialNB

#Create a Gaussian Classifier
NB = MultinomialNB()

# Train the model using the training sets
NB.fit(X_train, y_train)

#Predict Output
preds = NB.predict(X_test)
#preds_valid = lreg.predict(xvalid)
print("Acc = ", accuracy_score(preds, y_test))
print("F1-Macro",f1_score(y_test, preds,average='macro'))

First trained on 200k unlabeled F1-scores:
0.20333333333333334
0.10323744491762131

2nd trained on 300k unlabeled F1-scores:
0.22357142857142856
0.1309353180237549